In [1]:
import geopandas as gpd
import pandas as pd
import verde as vd
import numpy as np
from shapely import geometry
import my_funcs as mf

import matplotlib.pyplot as plt
%matplotlib widget

gdb='/home/ggrl/geodatabase/'

ModuleNotFoundError: No module named 'my_funcs'